In [ ]:
# Импорт библиотек
import os
import warnings
from pathlib import Path
from collections import OrderedDict
import glob
import random

import numpy as np
import pandas as pd

import librosa
import librosa.display
import soundfile as sf

import cv2
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    roc_curve,
    auc
)

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import (
    Dataset,
    DataLoader,
    Subset,
    random_split
)
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder

from tqdm import tqdm

warnings.filterwarnings('ignore')

torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

In [ ]:
if torch.cuda.is_available():
    device=torch.device("cuda:0")
    print("Training on GPU...")
else:
    device = torch.device("cpu")
    print("Training on CPU...")

In [ ]:
class MetricCalculator:
    def __init__(self, num_classes):
        self.num_classes = num_classes
        self.dice_metric = monai.metrics.DiceMetric(include_background=True, reduction="mean")

    def compute_metrics(self, y_pred, y_true):
        # Возвращает словарь
        metrics = {
            'loss': criterion(y_pred, y_true).item(),
            'accuracy': self._compute_accuracy(y_pred, y_true),
            'dice': self._compute_dice(y_pred, y_true),
            'confusion_matrix': self._compute_confusion_matrix(y_pred, y_true)
        }
        return metrics

    def _compute_accuracy(self, y_pred, y_true):
        with torch.no_grad():
            preds = torch.argmax(y_pred, dim=1)
            correct = (preds == y_true).float().sum()
            return (correct / y_true.shape[0]).item()

    def _compute_dice(self, y_pred, y_true):
        y_pred_onehot = torch.softmax(y_pred, dim=1)
        y_true_onehot = torch.nn.functional.one_hot(y_true, num_classes=self.num_classes).permute(0, 3, 1, 2)

        self.dice_metric(y_pred=y_pred_onehot, y_true=y_true_onehot)
        dice = self.dice_metric.aggregate().item()
        self.dice_metric.reset()
        return dice

    def _compute_confusion_matrix(self, y_pred, y_true):
        with torch.no_grad():
            preds = torch.argmax(y_pred, dim=1)
            return confusion_matrix(
                y_true.cpu().numpy(),
                preds.cpu().numpy(),
                labels=range(self.num_classes)
            )

In [ ]:
train_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(size=(150, 150)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

])


val_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(size=(150, 150)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225])

])


test_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(size=(150, 150)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225])

])

In [ ]:
shapes = []
for folder in glob.glob("/kaggle/working/dataset/*"):
    for file in glob.glob(f"{folder}/*"):
        shapes.append(cv2.imread(file).shape)
shapes = np.array(shapes)
shapes

In [ ]:
train_val_path="/kaggle/working/dataset"

train_dataset = ImageFolder(train_val_path, transform=train_transform)
val_dataset = ImageFolder(train_val_path, transform=val_transform)

class_names = train_dataset.classes
print("Class names:", class_names)

In [ ]:
def split_train_val(tot_img, val_percentage=0.2, rnd=23):
    number_of_val = int(tot_img*val_percentage)

    np.random.seed(rnd)
    indexs = np.random.permutation(tot_img)
    return indexs[0:number_of_val], indexs[number_of_val:]

randomness = 1
val_per = 0.2

all_len = len(train_dataset)

val_indices, train_indices = split_train_val(all_len, val_per, randomness)

print(val_indices, "validation data:", val_indices.shape)
print(train_indices, "train data:", train_indices.shape)

In [ ]:
train_dataset = Subset(train_dataset, train_indices)
val_dataset = Subset(val_dataset, val_indices)

In [ ]:
batch_size = 8

train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size, shuffle=False)

In [ ]:
def show_images(images, labels, preds):
    plt.figure(figsize=(10,10))

    images = images.cpu()
    labels = labels.cpu()
    preds = preds.cpu()

    for i, image in enumerate(images):
        plt.subplot(1, 8, i + 1, xticks = [], yticks =[])
        image = image.numpy().transpose((1, 2, 0))

        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = image * std + mean
        image = np.clip(image, 0., 1.)
        plt.imshow(image)

        col = 'green' if preds[i] == labels[i] else 'red'

        plt.xlabel(f'{class_names[int(labels[i].numpy())]}')
        plt.ylabel(f'{class_names[int(preds[i].numpy())]}', color=col)
    plt.tight_layout()
    plt.show()

In [ ]:
model = torchvision.models.densenet121(pretrained=True)

classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 512)),
                          ('silu1', nn.SiLU()),
                          ('fc2', nn.Linear(512, 256)),
                          ('silu2', nn.SiLU()),
                           ('fc3', nn.Linear(256, 1)),
                           ('sigmoid', nn.Sigmoid())
                           ]))

model.classifier = classifier

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)

        self.pool = nn.AdaptiveAvgPool2d((4, 4))

        self.fc1 = nn.Linear(256 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 1)

        self.dropout = nn.Dropout(0.2)

    def forward(self, x):

        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout(x)

        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout(x)

        x = F.relu(self.conv3(x))
        x = self.pool(x)

        x = x.view(x.size(0), -1)

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = torch.sigmoid(x)
        return x

model = SimpleCNN()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
model = model.to(device)

criterion = torch.nn.BCELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)

In [ ]:
def show_preds():
    model.eval()
    images, labels = next(iter(val_dataloader))

    with torch.no_grad():
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)

    preds = torch.round(outputs)
    show_images(images, labels, preds)

In [ ]:
def train(model, train_dataloader, val_dataloader, device, criterion, optimizer, epochs=30):
    """
    Обучает модель и оценивает ее на валидационном наборе.
    В конце обучения строит графики обучения (loss и accuracy).

    Args:
        model: Модель PyTorch для обучения.
        train_dataloader: DataLoader для обучающих данных.
        val_dataloader: DataLoader для валидационных данных.
        device: Устройство (cuda или cpu).
        criterion: Функция потерь.
        optimizer: Оптимизатор.
        epochs: Количество эпох для обучения.
    """

    print('Starting training..')
    train_losses = []
    val_losses = []
    val_accuracies = []

    for e in range(epochs):
        print('='*20)
        print(f'Starting epoch {e + 1}/{epochs}')
        print('='*20)

        train_loss = 0.
        val_loss = 0.

        model.train()

        for train_step, (images, labels) in enumerate(train_dataloader):
            images = images.to(device)
            labels = labels.to(device).float()

            optimizer.zero_grad()
            outputs = model(images)

            outputs = outputs.squeeze(1)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= (train_step + 1)
        train_losses.append(train_loss)
        print(f'Training Loss: {train_loss:.4f}')

        print('Starting validation')
        model.eval()
        accuracy = 0
        with torch.no_grad():
            for val_step, (images, labels) in enumerate(val_dataloader):
                images = images.to(device)
                labels = labels.to(device).float()

                outputs = model(images)
                outputs = outputs.squeeze(1)
                loss = criterion(outputs, labels)

                val_loss += loss.item()

                preds = torch.round(torch.sigmoid(outputs))
                accuracy += (preds == labels).sum().item()

        val_loss /= (val_step + 1)
        val_losses.append(val_loss)
        accuracy = accuracy / len(val_dataloader.dataset)
        val_accuracies.append(accuracy)
        print(f'Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.4f}')

    print('Training complete..')

    # === Построение графиков ===
    epochs_range = range(1, epochs + 1)

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, train_losses, label='Training Loss')
    plt.plot(epochs_range, val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, val_accuracies, label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Validation Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

train(model, train_dataloader, val_dataloader, device, criterion, optimizer)

In [ ]:
%%time

train(epochs=10)

In [ ]:
test_path="/kaggle/working/test"

test_dataset = ImageFolder(test_path, transform=test_transform)

class_names = test_dataset.classes
print("Class names:", class_names)

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns

def test(model, test_dataloader, device, criterion, class_names):
    """
    Выполняет тестирование модели на тестовых данных и выводит результаты.

    Args:
        model: Обученная модель PyTorch.
        test_dataloader: DataLoader для тестовых данных.
        device: Устройство для выполнения вычислений ("cuda" или "cpu").
        criterion: Функция потерь (например, nn.BCELoss()).
        class_names: Список названий классов.
    """

    model.eval()
    test_loss = 0.0
    all_preds = []
    all_labels = []
    all_probs = []
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device).float()

            outputs = model(inputs)
            outputs = outputs.squeeze(1)

            loss = criterion(outputs, labels)

            test_loss += loss.item()

            preds = torch.round(outputs)


            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(outputs.cpu().numpy())

            correct_predictions += (preds == labels).sum().item()
            total_samples += labels.size(0)


    test_loss /= len(test_dataloader)
    test_accuracy = correct_predictions / total_samples

    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

    # Вывод отчета о классификации
    print("\nTest Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))

    # Построение матрицы ошибок
    plot_confusion_matrix(all_labels, all_preds, class_names)

    # Построение ROC-кривой
    plot_roc_curve(all_labels, np.array(all_probs), class_names)

def plot_confusion_matrix(labels, preds, classes):
    """Строит и отображает матрицу ошибок."""
    cm = confusion_matrix(labels, preds)
    plt.figure(figsize=(len(classes), len(classes)))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()


def plot_roc_curve(labels, probs, classes):
    """Строит и отображает ROC-кривые для каждого класса."""
    plt.figure(figsize=(8, 6))
    n_classes = len(classes)
    labels = np.array(labels)

    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(labels, probs)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{classes[i]} (AUC = {roc_auc:.2f})')

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
  # Запуск тестирования
test(
    model=model,
    test_dataloader=test_dataloader,
    device=device,
    criterion=torch.nn.BCELoss(),
    class_names=['1', '0']
)